# Metro Manila Flood Hazard Analysis using NOAH Dataset

## Overview

The Philippines is highly vulnerable to the effects of natural disasters. The latest [World Risk Index 2023](https://weltrisikobericht.de/wp-content/uploads/2024/01/WorldRiskReport_2023_english_online.pdf) assessment ranked the Philippines with the highest disaster risk worldwide. Geographically located in the Pacific Ring of Fire, the country is prone to earthquakes, volcanic eruptions, and notably typhoons. The [DOST-PAGASA ](https://www.pagasa.dost.gov.ph/climate/tropical-cyclone-information) stated that the country experiences an average of 20 typhoons annually with 8 to 9 making landfall.

Typhoons often lead to significant flooding, particularly in low-lying and coastal areas, which displaces thousands of people and leads to loss of life and property. Early warning systems, disaster risk reduction programs, and coping facilities are necessary to strengthen disaster preparedness measures. One such program is **Project NOAH** (Nationwide Operational Assessment of Hazard), a disaster prevention and mitigation system that includes geo-hazard vulnerability maps that identify flood-prone areas throughout the country. 

We use the UP Resilience Institute (UPRI) [NOAH Open Datasets](https://drive.google.com/drive/folders/1K2z1FYNCGPzhcGIgT1Y2nKKLUojxxIa3) to analyze the hazard level of cities across Metro Manila.

## Libraries

We import relevant Python libraries to conduct our data analysis.

In [21]:
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import folium

print("Libraries are imported!")

Libraries are imported!


## The Dataset 

We use the 100-year return period data of Metro Manila.

In [22]:
# Load the flood hazard shapefile data
file_path = "MetroManila/MetroManila_Flood_100year.shp"
gdf = gpd.read_file(file_path)

# Ensure the geometry column is properly set
gpd.GeoDataFrame(gdf, geometry='geometry')

,Var,geometry
0,1.0,"MULTIPOLYGON (((285550.000 1590130.000, 285550..."
1,2.0,"MULTIPOLYGON (((285590.000 1590210.000, 285590..."
2,3.0,"MULTIPOLYGON (((287455.000 1590365.000, 287465..."


Observe that our data only has 3 rows and 2 columns. The rows represent the flood hazard levels: low (Var=1.0), medium (Var=2.0), and high (Var=3.0). From the [NOAH Flood Hazard Maps Metadata](https://drive.google.com/file/d/1qbOoN_CZ6AUIk22owtJxRFENALIWCszq/view), the ```Var``` column is the hazard classification indicator with the following flood depths:
1. Low hazard (0-0.5 meters)
2. Medium hazard (>0.5-1.5 meters)
3. High hazard (>1.5 meters)

The ```geometry``` column contains the polygon data for flood hazard map. For now, we use ```EPSG:4326```.

Let us separate each flood hazard level.

In [23]:
# We use EPSG:4326
my_epsg = 4326

# Convert gdf to match our EPSG
gdf = gdf.to_crs(crs=my_epsg)

# Flood Hazard Levels
low = gdf.iloc[[0]]
med = gdf.iloc[[1]]
high = gdf.iloc[[2]]

To visualize the data much easier, let us try and plot hazard levels with Metro Manila as the basemap.

In [24]:
# # Create the basemap for the data
# m = folium.Map(location=ox.geocode("Metro Manila, Philippines"), zoom_start=11, tiles="CartoDb DarkMatter")

# # Add the hazard levels separately with appropriate colors
# folium.GeoJson(low, 
#                style_function=lambda x: 
#                    {'color': 'yellow', 
#                     'weight': 0, 
#                     'fillOpacity': 0.6}).add_to(m)

# folium.GeoJson(med, 
#                style_function=lambda x: 
#                    {'color': 'orange', 
#                     'weight': 0, 
#                     'fillOpacity': 0.6}).add_to(m)

# folium.GeoJson(high, 
#                style_function=lambda x: 
#                    {'color': 'red', 
#                     'weight': 0, 
#                     'fillOpacity': 0.6}).add_to(m)

# # Save the map
# # m.save("Outputs/Metro_Manila_Hazard_Map.html")

# # Display the map
# m

Note that the following colors are used to represent the flood hazard levels.
1. **Low** - Yellow
2. **Medium** - Orange
3. **High** - Red

We use this color scheme througout the analysis. Observe that the hazard classifications vary depending throughout Metro Manila.

## Flood Hazard Maps of Cities in Metro Manila

We use [Nominatim](https://nominatim.openstreetmap.org/ui/search.html), a search engine for **OpenStreetMap** (OSM) data that allows users to convert addresses into geographic coordinates (geocoding). This allows us to obtain the necessary geographic coordinates for the cities in Metro Manila.

Let us create the list of address of the cities in Metro Manila.

In [25]:
# Nominatim address of cities in Metro Manila
cities = [
    "Caloocan, Northern Manila District, Metro Manila, Philippines",
    "Las Piñas, Southern Manila District, Metro Manila, Philippines",
    "Makati, Southern Manila District, Metro Manila, Philippines",
    "Malabon, Northern Manila District, Metro Manila, Philippines",
    "Mandaluyong, Eastern Manila District, Metro Manila, Philippines",
    "Manila, Capital District, Metro Manila, Philippines",
    "Marikina, Eastern Manila District, Metro Manila, Philippines",
    "Muntinlupa, Southern Manila District, Metro Manila, Philippines",
    "Navotas, Northern Manila District, Metro Manila, Philippines",
    "Parañaque, Southern Manila District, Metro Manila, 1700, Philippines",
    "Pasay, Southern Manila District, Metro Manila, Philippines",
    "Pasig, Eastern Manila District, Metro Manila, Philippines",
    "Quezon City, Eastern Manila District, Metro Manila, Philippines",
    "San Juan, Eastern Manila District, Metro Manila, Philippines",
    "Taguig, Southern Manila District, Metro Manila, Philippines",
    "Valenzuela, Northern Manila District, Metro Manila, Philippines"
]

Let us create some necessary ```GeoDataFrame```, so that we can easily select needed geometry.

In [26]:
# Create a GeoDataFrame for the boundary of each city
city_boundary = gpd.GeoDataFrame(
    {
        "address": cities,
        "geometry": [ox.geocode_to_gdf(city).geometry.iloc[0] for city in cities]
    }
).set_crs(epsg=my_epsg)

# Function to get geometries from geocode and clip data
def get_city_hazard(city, hazard_data):
    city_geom = city_boundary.geometry[city_boundary['address'] == city].iloc[0]
    return gpd.clip(hazard_data, city_geom).geometry.iloc[0]

# Create GeoDataFrames for each hazard level
low_hazard_city = gpd.GeoDataFrame(
    {
        "address": cities,
        "geometry": [get_city_hazard(city, low) for city in cities]
    }
).set_crs(epsg=my_epsg)

med_hazard_city = gpd.GeoDataFrame(
    {
        "address": cities,
        "geometry": [get_city_hazard(city, med) for city in cities]
    }
).set_crs(epsg=my_epsg)

high_hazard_city = gpd.GeoDataFrame(
    {
        "address": cities,
        "geometry": [get_city_hazard(city, high) for city in cities]
    }
).set_crs(epsg=my_epsg)

Let us define ```city_hazard_map``` function to visualize flood hazard maps of the cities.

In [27]:
def city_hazard_map(city, low=True, med=True, high=True, save_file=False):
    """
    Description: Returns a Folium map depending on the address parameter.

    Parameters:
        - city *(str): Specify the address to see the flood hazard map of the city of interest
        - low (bool): Defeault True. If False, the map will not show low flood hazard polygons
        - med (bool): Similar to low
        - high (bool): Similar to low
        - save_file (bool): Default False. If True, save the hazard map in 'Outputs/Cities'
    """
    # Get the place boundary geometry as a GeoDataFrame
    boundaries_gdf = city_boundary.loc[city_boundary.address == city]
    
    # Flood Hazard Geometries
    low_haz = low_hazard_city.loc[low_hazard_city.address == city]
    med_haz = med_hazard_city.loc[med_hazard_city.address == city]
    high_haz = high_hazard_city.loc[high_hazard_city.address == city]
    
    # Create the city basemap
    city_map = folium.Map(location=ox.geocode(city), zoom_start=12, tiles="CartoDb DarkMatter")
    
    # Add the hazard levels separately with appropriate colors
    if low:
        folium.GeoJson(low_haz, 
                       style_function=lambda x: 
                           {'color': 'yellow', 
                            'weight': 0, 
                            'fillOpacity': 0.6}).add_to(city_map)
        
    if med:
        folium.GeoJson(med_haz, 
                       style_function=lambda x: 
                           {'color': 'orange', 
                            'weight': 0, 
                            'fillOpacity': 0.6}).add_to(city_map)

    if high:
        folium.GeoJson(high_haz, 
                       style_function=lambda x: 
                           {'color': 'red', 
                            'weight': 0, 
                            'fillOpacity': 0.6}).add_to(city_map)

    # Add the boundary geometry of the city
    folium.GeoJson(boundaries_gdf,
                   style_function=lambda x:
                       {'color': 'white',
                        'weight': 1,
                       'fillOpacity': 0.0}).add_to(city_map)

    if save_file:
        file_name = f"Outputs/Cities/{city.split(',')[0].strip().replace(' ', '_').title()}_Hazard_Map.html"
        city_map.save(file_name)
    
    # Return the city hazard map
    return city_map

To see all the flood hazard maps of cities in Metro Manila, you can check out ```Outputs/Cities``` directory.

In [28]:
# Save all flood hazard map of Metro Manila Cities
# for city in cities:
    # city_hazard_map(city, save_file=True)

## Flood Hazard Ranking of Cities in Metro Manila

Observe that the flood hazard maps vary depending on the cities. We can quantify these variations *(and hence rank the cities)* by using the formula below: $$\text{Flood Hazard Level of the City} = \frac{\text{total area of the flood hazard} \, \text{(km)}^2}{\text{total area of the city} \, \text{(km)}^2}.$$ 

Note that ```my_epsg=4326``` which is not accurate in measuring area. For this, we use ```area_epsg=3832``` which gives the area calculation (in $\text{m}^2$).

In [29]:
# Area EPSG 
area_epsg=3832

# Areas of each cities in km^2
city_area = [
    54.4,  # Caloocan
    38.5,  # Las Piñas
    27.36, # Makati
    38.3,  # Malabon
    16.3,  # Mandaluyong
    42.88, # Manila
    21.5,  # Marikina
    45.3,  # Muntinlupa
    10.9,  # Navotas
    47.2,  # Parañaque
    18.5,  # Pasay
    31.5,  # Pasig
    166.2, # Quezon City
    5.9,   # San Juan
    47.6,  # Taguig
    45.8   # Valenzuela
]

Using ```sort_values()``` we can rank each city based on their total area.

In [30]:
# Create a new GeoDataFrame for the rankings
city_ranking = city_boundary.copy()

# Drop the geometry column
city_ranking = city_ranking.drop(columns=['geometry'])

# Add the area of each city
city_ranking['land_area'] = city_area

# Sort them by most to least covered land area
city_ranking.sort_values(by='land_area', ascending=False)

,address,land_area
12,"Quezon City, Eastern Manila District, Metro Ma...",166.20
0,"Caloocan, Northern Manila District, Metro Mani...",54.40
14,"Taguig, Southern Manila District, Metro Manila...",47.60
9,"Parañaque, Southern Manila District, Metro Man...",47.20
15,"Valenzuela, Northern Manila District, Metro Ma...",45.80
7,"Muntinlupa, Southern Manila District, Metro Ma...",45.30
5,"Manila, Capital District, Metro Manila, Philip...",42.88
1,"Las Piñas, Southern Manila District, Metro Man...",38.50
3,"Malabon, Northern Manila District, Metro Manil...",38.30
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50


We see that Quezon City has the highest land area while San Juan has the lowest.

### Low Flood Hazard

We can start with low flood hazard area.

In [31]:
low_hazard_city = low_hazard_city.to_crs(crs=area_epsg)
city_ranking['low_area'] = low_hazard_city.geometry.area/1_000_000
city_ranking.sort_values(by='low_area', ascending=False)

,address,land_area,low_area
12,"Quezon City, Eastern Manila District, Metro Ma...",166.20,16.063475
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912
14,"Taguig, Southern Manila District, Metro Manila...",47.60,6.826070
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719
1,"Las Piñas, Southern Manila District, Metro Man...",38.50,5.973249
7,"Muntinlupa, Southern Manila District, Metro Ma...",45.30,4.742067
10,"Pasay, Southern Manila District, Metro Manila,...",18.50,4.231235
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168
2,"Makati, Southern Manila District, Metro Manila...",27.36,4.012196
8,"Navotas, Northern Manila District, Metro Manil...",10.90,3.631271


We see that the top city with the most low flood hazard covered area is Quezon City, followed by Parañaque, Taguig, Manila, and Las Piñas. The one with the least, however, is San Juan. 

Let us see their low flood hazard map to verify this.

In [32]:
# # Get the top and bottom in the ranking
# top_low = city_ranking.sort_values(by='low_area', ascending=False).iloc[0,0]
# bottom_low = city_ranking.sort_values(by='low_area', ascending=True).iloc[0,0]

# # Show the low flood hazard map for both cities
# print(f"Top Low Flood Hazard Map: {top_low}")
# display(city_hazard_map(top_low, low=True, med=False, high=False))
# print("")
# print(f"Bottom Low Flood Hazard Map: {bottom_low}")
# display(city_hazard_map(bottom_low, low=True, med=False, high=False))

### Medium Flood Hazard

We continue with the medium flood hazard.

In [33]:
med_hazard_city = med_hazard_city.to_crs(crs=area_epsg)
city_ranking['med_area'] = med_hazard_city.geometry.area/1_000_000
city_ranking.sort_values(by='med_area', ascending=False)

,address,land_area,low_area,med_area
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719,22.017171
12,"Quezon City, Eastern Manila District, Metro Ma...",166.20,16.063475,18.327390
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168,10.360062
15,"Valenzuela, Northern Manila District, Metro Ma...",45.80,1.222812,10.250230
14,"Taguig, Southern Manila District, Metro Manila...",47.60,6.826070,9.840910
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912,9.162374
3,"Malabon, Northern Manila District, Metro Manil...",38.30,2.025560,7.950973
1,"Las Piñas, Southern Manila District, Metro Man...",38.50,5.973249,6.376223
0,"Caloocan, Northern Manila District, Metro Mani...",54.40,3.559701,5.913534
10,"Pasay, Southern Manila District, Metro Manila,...",18.50,4.231235,5.555206


We see that the top city with the most medium flood hazard covered area is Manila, followed by Quezon City, Pasig, Valenzuela, and Taguig. The one with the least is still San Juan.

### High Flood Hazard

In [34]:
high_hazard_city = high_hazard_city.to_crs(crs=area_epsg)
city_ranking['high_area'] = high_hazard_city.geometry.area/1_000_000
city_ranking.sort_values(by='high_area', ascending=False)

,address,land_area,low_area,med_area,high_area
12,"Quezon City, Eastern Manila District, Metro Ma...",166.20,16.063475,18.327390,20.328218
6,"Marikina, Eastern Manila District, Metro Manil...",21.50,1.027355,3.866713,11.787710
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168,10.360062,8.678232
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719,22.017171,5.446242
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912,9.162374,4.197007
15,"Valenzuela, Northern Manila District, Metro Ma...",45.80,1.222812,10.250230,4.081427
3,"Malabon, Northern Manila District, Metro Manil...",38.30,2.025560,7.950973,3.372452
0,"Caloocan, Northern Manila District, Metro Mani...",54.40,3.559701,5.913534,2.962203
1,"Las Piñas, Southern Manila District, Metro Man...",38.50,5.973249,6.376223,2.035532
14,"Taguig, Southern Manila District, Metro Manila...",47.60,6.826070,9.840910,1.968934


We see that the top city with the most high flood hazard covered area is Quezon City, followed by Marikina, Pasig, Manila, and Parañaque. The one with the least, however, is Navotas.

### Flood Hazard Percentage against Land Area

However, these rankings do not complete the whole picture for us. To give us more insights let us try to get the ratio of these values against the total area of the city.

In [35]:
city_ranking['low_ratio'] = city_ranking['low_area']/city_ranking['land_area']*100
city_ranking.sort_values(by='low_ratio', ascending=False)

,address,land_area,low_area,med_area,high_area,low_ratio
8,"Navotas, Northern Manila District, Metro Manil...",10.90,3.631271,3.481383,0.258076,33.314416
10,"Pasay, Southern Manila District, Metro Manila,...",18.50,4.231235,5.555206,1.048267,22.871541
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912,9.162374,4.197007,16.504050
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719,22.017171,5.446242,15.670987
1,"Las Piñas, Southern Manila District, Metro Man...",38.50,5.973249,6.376223,2.035532,15.514932
2,"Makati, Southern Manila District, Metro Manila...",27.36,4.012196,4.330344,0.798625,14.664460
14,"Taguig, Southern Manila District, Metro Manila...",47.60,6.826070,9.840910,1.968934,14.340482
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168,10.360062,8.678232,12.927516
13,"San Juan, Eastern Manila District, Metro Manil...",5.90,0.665825,0.737673,0.930162,11.285176
7,"Muntinlupa, Southern Manila District, Metro Ma...",45.30,4.742067,4.407102,1.443135,10.468138


Earlier, we said that the top city with the most low flood hazard covered area is Quezon City. After comparing the covered area against the total land area of the city, we now see that Navotas has the highest low flood hazard area covering around ```33.3%``` of its total land area. The lowest, however, is Valenzuela with only ```2.7%``` of its area covered in low flood hazard.

Let us try and see the two cities to verify this analysis.

In [36]:
# Get the top and bottom in the ranking
# top_low = city_ranking.sort_values(by='low_ratio', ascending=False).iloc[0,0]
# bottom_low = city_ranking.sort_values(by='low_ratio', ascending=True).iloc[0,0]

# # Show the low flood hazard map for both cities
# print(f"Top Low Flood Hazard Map: {top_low}")
# display(city_hazard_map(top_low, low=True, med=False, high=False))
# print("")
# print(f"Bottom Low Flood Hazard Map: {bottom_low}")
# display(city_hazard_map(bottom_low, low=True, med=False, high=False))

By simple inspection, we see that our result is verified by the low flood hazard map of these two cities. We continue on with medium and high flood hazards

In [37]:
city_ranking['med_ratio'] = city_ranking['med_area']/city_ranking['land_area']*100
city_ranking.sort_values(by='med_ratio', ascending=False)

,address,land_area,low_area,med_area,high_area,low_ratio,med_ratio
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719,22.017171,5.446242,15.670987,51.346015
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168,10.360062,8.678232,12.927516,32.889084
8,"Navotas, Northern Manila District, Metro Manil...",10.90,3.631271,3.481383,0.258076,33.314416,31.939296
10,"Pasay, Southern Manila District, Metro Manila,...",18.50,4.231235,5.555206,1.048267,22.871541,30.028143
15,"Valenzuela, Northern Manila District, Metro Ma...",45.80,1.222812,10.250230,4.081427,2.669895,22.380415
3,"Malabon, Northern Manila District, Metro Manil...",38.30,2.025560,7.950973,3.372452,5.288668,20.759722
14,"Taguig, Southern Manila District, Metro Manila...",47.60,6.826070,9.840910,1.968934,14.340482,20.674181
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912,9.162374,4.197007,16.504050,19.411810
6,"Marikina, Eastern Manila District, Metro Manil...",21.50,1.027355,3.866713,11.787710,4.778394,17.984710
1,"Las Piñas, Southern Manila District, Metro Man...",38.50,5.973249,6.376223,2.035532,15.514932,16.561619


In [38]:
city_ranking['high_ratio'] = city_ranking['high_area']/city_ranking['land_area']*100
city_ranking.sort_values(by='high_ratio', ascending=False)

,address,land_area,low_area,med_area,high_area,low_ratio,med_ratio,high_ratio
6,"Marikina, Eastern Manila District, Metro Manil...",21.50,1.027355,3.866713,11.787710,4.778394,17.984710,54.826560
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168,10.360062,8.678232,12.927516,32.889084,27.549942
13,"San Juan, Eastern Manila District, Metro Manil...",5.90,0.665825,0.737673,0.930162,11.285176,12.502932,15.765456
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719,22.017171,5.446242,15.670987,51.346015,12.701123
12,"Quezon City, Eastern Manila District, Metro Ma...",166.20,16.063475,18.327390,20.328218,9.665147,11.027311,12.231178
15,"Valenzuela, Northern Manila District, Metro Ma...",45.80,1.222812,10.250230,4.081427,2.669895,22.380415,8.911414
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912,9.162374,4.197007,16.504050,19.411810,8.891964
3,"Malabon, Northern Manila District, Metro Manil...",38.30,2.025560,7.950973,3.372452,5.288668,20.759722,8.805358
10,"Pasay, Southern Manila District, Metro Manila,...",18.50,4.231235,5.555206,1.048267,22.871541,30.028143,5.666309
0,"Caloocan, Northern Manila District, Metro Mani...",54.40,3.559701,5.913534,2.962203,6.543568,10.870468,5.445226


Let us assign ```penalty``` values for each floow hazard level.

In [39]:
# Penalties for Risk Scores
low_penalty = 1.3
med_penalty = 1.5
high_penalty = 1.8

Note that flood hazards vary in intensity, with high-risk areas being more prone to severe flooding. These penalties help us differentiate between areas of varying vulnerability, thus capturing the gradations in risk levels rather than treating all risks equally.

These penalties are assigned arbitrarily and other values may be used as well. Let us create a new column ```risk_score``` by following the formula below $$\text{Risk Score} = \sum_{i=1}^3 \, \lambda_i A_i$$ where $A_i$ is the area for low, medium, and high flood hazard while $\lambda_i$ is the penalty for each hazard level.

In [40]:
# Caculate the Risk Score for each city
city_ranking['risk_score'] = (city_ranking['low_area'] * low_penalty) + (city_ranking['med_area'] * med_penalty) + (city_ranking['high_area'] * high_penalty)

city_ranking.sort_values(by='risk_score', ascending=False)

,address,land_area,low_area,med_area,high_area,low_ratio,med_ratio,high_ratio,risk_score
12,"Quezon City, Eastern Manila District, Metro Ma...",166.20,16.063475,18.327390,20.328218,9.665147,11.027311,12.231178,84.964395
5,"Manila, Capital District, Metro Manila, Philip...",42.88,6.719719,22.017171,5.446242,15.670987,51.346015,12.701123,51.564627
11,"Pasig, Eastern Manila District, Metro Manila, ...",31.50,4.072168,10.360062,8.678232,12.927516,32.889084,27.549942,36.454727
9,"Parañaque, Southern Manila District, Metro Man...",47.20,7.789912,9.162374,4.197007,16.504050,19.411810,8.891964,31.425059
6,"Marikina, Eastern Manila District, Metro Manil...",21.50,1.027355,3.866713,11.787710,4.778394,17.984710,54.826560,28.353509
14,"Taguig, Southern Manila District, Metro Manila...",47.60,6.826070,9.840910,1.968934,14.340482,20.674181,4.136417,27.179338
15,"Valenzuela, Northern Manila District, Metro Ma...",45.80,1.222812,10.250230,4.081427,2.669895,22.380415,8.911414,24.311570
1,"Las Piñas, Southern Manila District, Metro Man...",38.50,5.973249,6.376223,2.035532,15.514932,16.561619,5.287097,20.993517
3,"Malabon, Northern Manila District, Metro Manil...",38.30,2.025560,7.950973,3.372452,5.288668,20.759722,8.805358,20.630101
0,"Caloocan, Northern Manila District, Metro Mani...",54.40,3.559701,5.913534,2.962203,6.543568,10.870468,5.445226,18.829879


We can make several implications revealed by the ```risk_score```:
1. Quezon City is the most flood risk city in Metro Manila followed by Manila.
2. San Juan is the least flood risk city in Metro Manila.
3. While Marikina has the most high flood hazard level, it is not most flood risk city.
4. While Valenzuela has the least low flood hazard level, it is not the least flood risk city.

However, it is important to note that the ```risk_score``` for each city depends only on one parameter, i.e., flood hazard levels. Other factors such as **exposure**, **vulnerability**, **coping facilities**, etc. may increase or decrease the ```risk_score```.

## Conclusion

This analysis highlights the severity of flood hazards in Metro Manila, a region highly susceptible to typhoon-induced flooding. Using the NOAH dataset’s 100-year return period flood maps, we classified areas into low, medium, and high hazard levels based on flood depth. This categorization provides a clearer view of flood-prone areas, supporting disaster risk planning and resource allocation.

The results can be instrumental for urban planners, policymakers, and emergency responders in prioritizing mitigation efforts and preparing infrastructure to withstand future flood events. Moving forward, integrating additional data layers, such as population density or infrastructure locations, could further enhance the granularity of hazard assessments, offering a comprehensive tool for flood resilience planning. 